## Work
1. 試比較有 BN 在 Batch_size = 2, 16, 32, 128, 256 下的差異
2. 請嘗試將 BN 放在 Activation 之前，並比較訓練結果
3. 請於 BN 放在 Input Layer 後，並比較結果

In [1]:
import os
import keras
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Activation
from keras.utils import to_categorical
import itertools

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)
input_shape = x_train.shape[1:]

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
def build_mlp1(input_shape, hidden_neuron_num, output_shape):
    input_layer = Input(shape=input_shape)
    for ind, num in enumerate(hidden_neuron_num):
        if ind == 0:
            x = Dense(units=num)(input_layer)
            x = BatchNormalization()(x)
            x = Activation(activation='relu')(x)
        else:
            x = Dense(units=num)(x)
            x = BatchNormalization()(x)
            x = Activation(activation='relu')(x)
            
    output_layer = Dense(units=output_shape, activation='softmax')(x)
    model = Model(inputs=[input_layer], outputs=[output_layer])
        
    return model

def build_mlp2(input_shape, hidden_neuron_num, output_shape):
    input_layer = Input(shape=input_shape)
    for ind, num in enumerate(hidden_neuron_num):
        if ind == 0:
            x = Dense(units=num)(input_layer)
            x = Activation(activation='relu')(x)
            x = BatchNormalization()(x)
        else:
            x = Dense(units=num)(x)
            x = Activation(activation='relu')(x)
            x = BatchNormalization()(x)
            
    output_layer = Dense(units=output_shape, activation='softmax')(x)
    model = Model(inputs=[input_layer], outputs=[output_layer])
        
    return model

In [6]:
## 超參數設定
LEARNING = 1e-4
EPOCHS = 5
BATCH_SIZE = [2, 16, 32, 128, 256]
HIDDEN_NEURON_NUM = [256, 128]
OUTPUT_SIZE = 10
# model 1 Batch-Normalization before activation
# model 2 Batch-Normalization before input layer
models = {'model1':build_mlp1, 'model2':build_mlp2}

In [ ]:
results = {}
for name, mlp in models.items():
    for size in BATCH_SIZE:
        keras.backend.clear_session()
        model = mlp(input_shape, HIDDEN_NEURON_NUM, OUTPUT_SIZE)
        model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        model.fit(x_train, y_train,
                  epochs=Epochs, 
                  batch_size=size,
                  validation_data=(x_test, y_test),
                  shuffle=True)
        
        train_loss = model.history.history["loss"]
        valid_loss = model.history.history["val_loss"]
        train_acc = model.history.history["acc"]
        valid_acc = model.history.history["val_acc"]
        
        experiment = ':'.join([name, str(size)])
        results[experiment] = {
            'train_loss': train_loss,
            'valid_loss': valid_loss,
            'train_acc': train_acc,
            'valid_acc': valid_acc
        }

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 107s 2ms/step - loss: 2.2379 - acc: 0.1699 - val_loss: 2.0945 - val_acc: 0.2520
Train on 50000 samples, validate on 10000 samples
Epoch 1/1
13776/50000 [=======>......................] - ETA: 12s - loss: 1.8392 - acc: 0.3481

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
color_bar = ["r", "g", "b", "y", "m", "k"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train_loss'])),results[cond]['train_loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid_loss'])),results[cond]['valid_loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.ylim([0, 5])
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train_acc'])),results[cond]['train_acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid_acc'])),results[cond]['valid_acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()